# Proyecto Final de Organización y Arquitectura de Computadora
## Interrupciones y Control de Procesos en el Desarrollo y Ejecución de Programa
### Ferrer, Sebastián. E-8-211301
### Lucero, Pedro. 8-973-1067

### Plan:
- Interrupción - Clase
- Ejecución - Clase
- anhadir_interrupcion
- ver_detalles (Tipo las puras interrupciones con cada T)
- calcular_resultado (Hacer las operaciones para los próximos dos)
- ver_resultado (Ver la tabla de control de procesos desarrollada)
- ver_bitacora

In [26]:
from collections import deque # lo usamos para implementar la pila de interrupciones

In [27]:
class Interrupcion:
    def __init__(self, IRQ, prioridad, funcion):
        self.IRQ = IRQ
        self.prioridad = prioridad
        self.funcion = funcion
    def __repr__(self):
        return f"I({self.funcion})"

In [28]:
def_irqs = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
def_prios = [1, 2, 11, 12, 13, 14, 15, 3, 4, 5, 6, 7, 8, 9, 10]
def_funcs = ["Reloj del sistema", 
             "Teclado",
             "COM2 y COM4",
             "COM1 y COM3",
             "Libre(5)",
             "Controlador Floppy - Diskette",
             "Puerto Paralelo - Impresora",
             "Reloj (tics) en tiempo real CMOS",
             "Red, sonido, puerto SCSI",
             "Libre(10)",
             "Libre(11)",
             "PS-mouse",
             "Co-procesador matemático",
             "Canal IDE primario(Disco)",
             "Libre(15)"]

interrupciones = {irq: Interrupcion(irq, prio, func)
                  for irq, prio, func in zip(def_irqs, def_prios, def_funcs)}

In [29]:
class Ejecucion:
    def __init__(self, duracion_programa, T_inicial):
        self.duracion_programa = duracion_programa
        self.interrupciones = [] # este es un arreglo de tuplas para manejar T's con duraciones
        self.T_inicial = T_inicial
        self.control_procesos = []
        self.bitacora = []
        # print(self.duracion_programa)
    
    def anhadir_interrupcion(self, timestamp, irq, duracion): # Añadimos las interrupciones y ordenamos
        # Interrupciones: (Timestamp redondeado a 2 decimales | IRQ de la int. | duración (en seg.) de la int.)
        self.interrupciones.append((round(timestamp, 2), interrupciones[irq], duracion))
        self.interrupciones.sort(key=lambda int: int[0])

    def ver_resultado(self):
        for nombre, T_o, T_f, _, _ in self.control_procesos:
            espacio_res = len(max(def_funcs, key=lambda x: len(x)))
            print(nombre.center(espacio_res), end="|")
            print(f"T={T_o}".center(5), end=" | ")
            print(f"T={T_f}".center(5))
        
    def ver_bitacora(self, tiempos):
        tiempos.sort()
        bitacora = []
        for pos, proceso in enumerate(self.control_procesos):
            for T in tiempos:
                if T > proceso[2] or T < proceso[1]:
                    continue
                bitacora.append([T, proceso[0], proceso[3], f"{proceso[1]} a {proceso[2]}", proceso[4]])
            
        espacio_res = len(max(def_funcs, key=lambda x: len(x)))
        print("T".center(3), end=" | ")
        print("Funcion".center(espacio_res), end=" | ")
        print("I.", end=" | ")
        print("Rango T".center(7), end=" | ")
        print("TR".center(2))
        for fila in bitacora:
            print(f"{fila[0]}".center(3), end=" | ")
            print(f"{fila[1]}".center(espacio_res), end=" | ")
            print(fila[2], end=" | ")
            print(f"{fila[3]}".center(7), end=" | ")
            print(f"{fila[4]}".center(2))
              
    def calcular_resultado(self):
        pila = deque() # pila: [prioridad, t_restante]
        pila.append([999, self.duracion_programa, "Programa"])
        
        control_procesos = []
        
        T = self.T_inicial
        int_index = 0
        while len(pila) > 0 and int_index < len(self.interrupciones):
            actual = pila.pop()
            sig_T = self.interrupciones[int_index][0]
            sig_prio = self.interrupciones[int_index][1].prioridad
            sig_name = self.interrupciones[int_index][1].funcion
            sig_t_res = self.interrupciones[int_index][2]
            siguiente = [sig_prio, sig_t_res, sig_name]
            int_index += 1
            
            delta_T = sig_T - T
            
            if actual[1] - delta_T <= 0:
                prev_T = T
                T += actual[1]
                int_index -= 1
                control_procesos.append((actual[2], prev_T, T, "No", 0))
                continue
            
            actual[1] -= delta_T
            
            prev_T = T
            T += delta_T
            control_procesos.append((actual[2], prev_T, T, "Si", actual[1]))
            
            # 1. Prio sig < actual → sig. "gana"
            if siguiente[0] < actual[0]:
                pila.append(actual)
                pila.append(siguiente)
            
            # 2. Prio sig > actual → actual "gana"
            if siguiente[0] > actual[0]:
                temp_q = deque()
                temp_q.append(actual)
                while pila[-1][0] < siguiente[0]:
                    temp_q.append(pila.pop())
                temp_q.append(siguiente)
                while temp_q:
                    pila.append(temp_q.pop())
        
        
        while len(pila) > 0:
            actual = pila.pop()
            prev_T = T
            T += actual[1]
            control_procesos.append((actual[2], prev_T, T, "No", 0))
        
        proc_index = 0
        while proc_index+1 < len(control_procesos):
            proceso = control_procesos[proc_index]
            sig_proc = control_procesos[proc_index+1]
            
            if proceso[0] == sig_proc[0]:
                proceso = (proceso[0], proceso[1], sig_proc[2], sig_proc[3], sig_proc[4])
                self.control_procesos.append(proceso)
                proc_index += 2
                continue
            
            if proceso[1] == proceso[2]:
                proc_index += 1
                continue
            
            self.control_procesos.append(proceso)
            proc_index += 1
        self.control_procesos.append(control_procesos[proc_index])

### Prueba pre-armada #1:
- Duracion de programa = 15
- 5 interrupciones:
  - T=3 | disco | 5seg
  - T=5 | red | 10seg
  - T=10 | impresora | 8seg
  - T=24 | disco | 12seg
  - T=42 | red | 5seg
- T final debe ser de: 55

In [30]:
# Prueba 1 pre armada, con lo definido
prueba1 = Ejecucion(15, 0)
prueba1.anhadir_interrupcion(3, 14, 5)
prueba1.anhadir_interrupcion(5, 9, 10)
prueba1.anhadir_interrupcion(10, 7, 8)
prueba1.anhadir_interrupcion(24, 14, 12)
prueba1.anhadir_interrupcion(42, 9, 5)
prueba1.interrupciones

[(3, I(Canal IDE primario(Disco)), 5),
 (5, I(Red, sonido, puerto SCSI), 10),
 (10, I(Puerto Paralelo - Impresora), 8),
 (24, I(Canal IDE primario(Disco)), 12),
 (42, I(Red, sonido, puerto SCSI), 5)]

In [31]:
prueba1.calcular_resultado()
prueba1.ver_resultado()

            Programa            | T=0  |  T=3 
   Canal IDE primario(Disco)    | T=3  |  T=5 
    Red, sonido, puerto SCSI    | T=5  |  T=15
   Canal IDE primario(Disco)    | T=15 |  T=18
  Puerto Paralelo - Impresora   | T=18 |  T=24
   Canal IDE primario(Disco)    | T=24 |  T=36
  Puerto Paralelo - Impresora   | T=36 |  T=38
            Programa            | T=38 |  T=42
    Red, sonido, puerto SCSI    | T=42 |  T=47
            Programa            | T=47 |  T=55


In [32]:
prueba1.ver_bitacora([8, 17, 20, 39, 45, 50])

 T  |             Funcion              | I. | Rango T | TR
 8  |     Red, sonido, puerto SCSI     | No |  5 a 15 | 0 
 17 |    Canal IDE primario(Disco)     | No | 15 a 18 | 0 
 20 |   Puerto Paralelo - Impresora    | Si | 18 a 24 | 2 
 39 |             Programa             | Si | 38 a 42 | 8 
 45 |     Red, sonido, puerto SCSI     | No | 42 a 47 | 0 
 50 |             Programa             | No | 47 a 55 | 0 


### Prueba pre-armada #2:
- Duracion de programa = 30
- 5 interrupciones:
  - T=6 | COM 1 | 5seg
  - T=15 | impresora | 8seg
  - T=20 | disco | 10seg
  - T=32 | COM 1 | 4seg
  - T=36 | Teclado | 3seg
- T final debe ser de: 60

In [33]:
# Prueba 1 pre armada, con lo definido antes
prueba2 = Ejecucion(30, 0)
prueba2.anhadir_interrupcion(6, 4, 5)
prueba2.anhadir_interrupcion(15, 7, 8)
prueba2.anhadir_interrupcion(20, 14, 10)
prueba2.anhadir_interrupcion(32, 4, 4)
prueba2.anhadir_interrupcion(36, 1, 3)
prueba2.interrupciones

[(6, I(COM1 y COM3), 5),
 (15, I(Puerto Paralelo - Impresora), 8),
 (20, I(Canal IDE primario(Disco)), 10),
 (32, I(COM1 y COM3), 4),
 (36, I(Teclado), 3)]

In [34]:
prueba2.calcular_resultado()
prueba2.ver_resultado()

            Programa            | T=0  |  T=6 
          COM1 y COM3           | T=6  |  T=11
            Programa            | T=11 |  T=15
  Puerto Paralelo - Impresora   | T=15 |  T=20
   Canal IDE primario(Disco)    | T=20 |  T=30
  Puerto Paralelo - Impresora   | T=30 |  T=32
          COM1 y COM3           | T=32 |  T=36
            Teclado             | T=36 |  T=39
  Puerto Paralelo - Impresora   | T=39 |  T=40
            Programa            | T=40 |  T=60


In [35]:
prueba2.ver_bitacora([8, 12, 18, 25, 31, 35])

 T  |             Funcion              | I. | Rango T | TR
 8  |           COM1 y COM3            | No |  6 a 11 | 0 
 12 |             Programa             | Si | 11 a 15 | 20
 18 |   Puerto Paralelo - Impresora    | Si | 15 a 20 | 3 
 25 |    Canal IDE primario(Disco)     | No | 20 a 30 | 0 
 31 |   Puerto Paralelo - Impresora    | Si | 30 a 32 | 1 
 35 |           COM1 y COM3            | No | 32 a 36 | 0 


### Prueba en vivo:
- T inicial de = _
- Duracion de programa = _
- _ interrupciones:
  - T=_ | _ | _seg
  - T=_ | _ | _seg
  - T=_ | _ | _seg
  - T=_ | _ | _seg
  - T=_ | _ | _seg
- T final debe ser de: _

### Ver las funciones por IRQ

In [15]:
for irq, func in zip(def_irqs, def_funcs):
    print("IRQ:", irq, "| Función:", func)

IRQ: 0 | Función: Reloj del sistema
IRQ: 1 | Función: Teclado
IRQ: 3 | Función: COM2 y COM4
IRQ: 4 | Función: COM1 y COM3
IRQ: 5 | Función: Libre(5)
IRQ: 6 | Función: Controlador Floppy - Diskette
IRQ: 7 | Función: Puerto Paralelo - Impresora
IRQ: 8 | Función: Reloj (tics) en tiempo real CMOS
IRQ: 9 | Función: Red, sonido, puerto SCSI
IRQ: 10 | Función: Libre(10)
IRQ: 11 | Función: Libre(11)
IRQ: 12 | Función: PS-mouse
IRQ: 13 | Función: Co-procesador matemático
IRQ: 14 | Función: Canal IDE primario(Disco)
IRQ: 15 | Función: Libre(15)


### Cargar datos:
#### Inicializar Ejecución(Duración_Programa, T_inicial)

In [38]:
prueba_v = Ejecucion(10, 0)

### Cargar datos:
#### anhadir_interrupcion(Tiempo, IRQ, Duración)

In [39]:
prueba_v.anhadir_interrupcion(4, 4, 8)
prueba_v.anhadir_interrupcion(9, 13, 5)
prueba_v.anhadir_interrupcion(12, 6, 15)
prueba_v.anhadir_interrupcion(25, 9, 12)
prueba_v.anhadir_interrupcion(35, 4, 3)
prueba_v.anhadir_interrupcion(43, 13, 5)
prueba_v.interrupciones

[(4, I(COM1 y COM3), 8),
 (9, I(Co-procesador matemático), 5),
 (12, I(Controlador Floppy - Diskette), 15),
 (25, I(Red, sonido, puerto SCSI), 12),
 (35, I(COM1 y COM3), 3),
 (43, I(Co-procesador matemático), 5)]

### Calcular y mostrar control de procesos:

In [40]:
prueba_v.calcular_resultado()
prueba_v.ver_resultado()

            Programa            | T=0  |  T=4 
          COM1 y COM3           | T=4  |  T=9 
    Co-procesador matemático    | T=9  |  T=14
          COM1 y COM3           | T=14 |  T=17
 Controlador Floppy - Diskette  | T=17 |  T=25
    Red, sonido, puerto SCSI    | T=25 |  T=37
          COM1 y COM3           | T=37 |  T=40
 Controlador Floppy - Diskette  | T=40 |  T=43
    Co-procesador matemático    | T=43 |  T=48
 Controlador Floppy - Diskette  | T=48 |  T=52
            Programa            | T=52 |  T=58


### Mostrar bitácora ([arreglo_tiempos]):

In [41]:
prueba_v.ver_bitacora([7, 10, 23, 39, 45, 50])

 T  |             Funcion              | I. | Rango T | TR
 7  |           COM1 y COM3            | Si |  4 a 9  | 3 
 10 |     Co-procesador matemático     | No |  9 a 14 | 0 
 23 |  Controlador Floppy - Diskette   | Si | 17 a 25 | 7 
 39 |           COM1 y COM3            | No | 37 a 40 | 0 
 45 |     Co-procesador matemático     | No | 43 a 48 | 0 
 50 |  Controlador Floppy - Diskette   | No | 48 a 52 | 0 
